In [11]:
import spacy
import json
import numpy as np
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
from spacy.training.example import Example

nlp = spacy.load("en_core_web_sm")

In [12]:
 def load_mappings(dataset):
    f = open(dataset, 'r',encoding='utf-8')
    model= dict({})
    for line in f:
        tem=line.split()
        vtem=[]
        for i in range(1,len(tem)):
            vtem.append( float(tem[i].strip()))
        vec1=np.array(vtem)
        model.update({tem[0].strip():vec1})
    return model

In [36]:
my_text = "I want to buy a blue phone and 8G ram."
doc = nlp(my_text)

In [37]:
for token in doc:
    if ("dobj" in token.dep_):
        subtree = list(token.subtree)
        start = subtree[0].i
        end = subtree[-1].i + 1
        # return subtree(list), token(product), extract(string)
        extract = doc[start:end]
        product = token.text


In [40]:
product

'phone'

In [42]:
extract

a blue phone and 8G ram

In [17]:
#load w2v
load_model = load_mappings("glove.6B.300d.txt")

In [43]:
# get_category 
def get_category(product, my_list):
    if product in load_model:
        embeddings = []
        v1 = load_model[product.strip()]
        embeddings.append(v1)
        for category in my_list:
            embeddings.append(load_model[category.strip()])
        cos_sim = cosine_similarity([embeddings[0]], embeddings[1:])
        return my_list[np.argmax(cos_sim)]
    return 0

In [44]:
category_list = ["phone", "laptop","clothes"]

In [45]:
my_category = get_category(token.text, category_list)
my_category

'phone'

In [46]:
# Sau khi có category thì đi vào file json để lấy ra các danh mục của item attributes
with open('data/category-list.json') as f:
    data = json.load(f)

# Lấy ra tất cả các category
phones = [item for item in data if item['category'] == my_category]
for item in phones:
    attributes_json = item['item attributes']

list_attributes_json = list(attributes_json.keys())
# Cần đưa về chữ thường để tránh các trường hợp chữ hoa W2V không bắt được
list_attributes_json = [x.lower() for x in list_attributes_json]

list_attributes_json

['color', 'ram', 'price']

In [47]:
# Bước 3
doc = nlp(extract.text)
list_attributes_noun = []
list_attributes_adj = []
# lấy ra các adj, và noun không phải là product lưu vào 2 list
for token in doc:
    if token.pos_ == "ADJ":
        list_attributes_adj.append(token.text.lower())
    elif token.pos_ == "NOUN" and token.text != str(product):
        list_attributes_noun.append(token.text.lower())

In [48]:
print(list_attributes_json)
print(list_attributes_adj)

['color', 'ram', 'price']
['blue']


In [ ]:
def get_attributes_list(list_attributes_adj, list_attributes_json):
    get_attributes = {}
    for attributes_json in list_attributes_json:
        embeddings = []
        if attributes_adj in load_model:
            v1 = load_model[product.strip()]
            embeddings.append(v1)
            for attributes_adj in list_attributes_adj:
                if attributes_json in load_model:
                    embeddings.append(load_model[category.strip()])
                else:
                    list_attributes_json.remove(attributes_json)
        cos_sim = cosine_similarity([embeddings[0]], embeddings[1:])
        get_attributes.append(my_list[np.argmax(cos_sim)])
    return 0